In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

from prediction_flow.features import Number, Category, Sequence, Features
from prediction_flow.transformers.column import (
    StandardScaler, CategoryEncoder, SequenceEncoder)

from prediction_flow.pytorch.data import Dataset
from prediction_flow.pytorch import WideDeep, DeepFM, DNN, DIN, DIEN, AttentionGroup

from prediction_flow.pytorch.functions import fit, predict, create_dataloader_fn

In [2]:
train_df = pd.read_csv("./ml-1m/train.csv")
valid_df = pd.read_csv("./ml-1m/test.csv")

In [3]:
train_df.movieId = train_df.movieId.astype(str)
valid_df.movieId = valid_df.movieId.astype(str)

train_df.histHighRatedMovieIds = train_df.histHighRatedMovieIds.astype(str)
valid_df.histHighRatedMovieIds = valid_df.histHighRatedMovieIds.astype(str)

train_df.negHistMovieIds = train_df.negHistMovieIds.astype(str)
valid_df.negHistMovieIds = valid_df.negHistMovieIds.astype(str)

In [4]:
train_df.dtypes

userId                     int64
movieId                   object
ratting                    int64
timestamp                  int64
last                     float64
histHighRatedMovieIds     object
negHistMovieIds           object
gender                    object
age                        int64
occupation                 int64
zipCode                   object
title                     object
genres                    object
label                      int64
dtype: object

In [5]:
valid_df.dtypes

userId                     int64
movieId                   object
ratting                    int64
timestamp                  int64
last                     float64
histHighRatedMovieIds     object
negHistMovieIds           object
gender                    object
age                        int64
occupation                 int64
zipCode                   object
title                     object
genres                    object
label                      int64
dtype: object

In [6]:
train_df.head()

,userId,movieId,ratting,timestamp,last,histHighRatedMovieIds,negHistMovieIds,gender,age,occupation,zipCode,title,genres,label
0,1,48,5,978824351,1.0,1545|527|595|588|1|2355|2294|783|1566|1907,393|2562|3289|3442|2808|2259|295|1190|3321|3470,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance,1
1,2030,48,3,977809337,1.0,3928|1934|2099|3345|1013|1101|3524|3429|3751|2138,3142|1743|525|1746|3323|935|2183|3556|3343|1542,M,25,4,77345,Pocahontas (1995),Animation|Children's|Musical|Romance,0
2,4877,48,2,962767153,1.0,2078|1032|2081|616|364|2033|3034|2394|3745|3615,2354|747|3478|2638|311|1204|3370|807|29|1380,M,25,4,94703,Pocahontas (1995),Animation|Children's|Musical|Romance,0
3,2,1917,3,978300174,1.0,3418|349|1527|2353|1370|648|368|736|2002|1544,2117|2560|3480|3678|2798|1222|3872|3503|3913|3494,M,56,16,70072,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,0
4,5122,1917,5,962213528,1.0,3078|2858|2907|1584|2571|32|788|1573|2428|1580,1860|194|742|2379|1966|3397|3195|678|2240|2196,M,25,0,20009,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,1


In [7]:
valid_df.head()

,userId,movieId,ratting,timestamp,last,histHighRatedMovieIds,negHistMovieIds,gender,age,occupation,zipCode,title,genres,label
0,9,2294,4,978226678,1.0,590|480|2302|2268|529|349|524|16|1653|2278,876|1953|2644|3540|2794|90|3291|406|2282|2406,M,25,17,61614,Antz (1998),Animation|Children's,1
1,15,3510,5,978361393,1.0,1892|648|1422|1597|3489|1196|260|1210|3461|2115,322|135|1324|3928|2808|3220|2992|3937|652|3671,M,25,7,22903,Frequency (2000),Drama|Thriller,1
2,2165,3510,5,974982139,1.0,1095|2959|2416|3552|16|1266|3210|104|3147|3148,1936|1248|2241|985|3097|2282|3550|2690|3731|718,M,25,7,32836,Frequency (2000),Drama|Thriller,1
3,2319,3510,4,974476547,1.0,2987|1240|2243|1302|2716|1961|1096|3408|3481|3148,392|2782|1615|3720|781|1511|3225|2273|2655|3798,M,35,7,48104,Frequency (2000),Drama|Thriller,1
4,16,2701,2,978174795,1.0,2683|2699|2355|2975|2369|3175|2761|2581|2394|2724,3128|1830|3281|163|1334|585|3580|1776|1392|774,F,35,0,20670,Wild Wild West (1999),Action|Sci-Fi|Western,0


In [8]:
print(train_df.shape, valid_df.shape)

(4832, 14) (1208, 14)


In [9]:
number_features = [
    Number('age', StandardScaler())
]

category_features = [
    Category('gender', CategoryEncoder(min_cnt=1)),
    Category('movieId', CategoryEncoder(min_cnt=1)),
    Category('occupation', CategoryEncoder(min_cnt=1)),
    Category('zipCode', CategoryEncoder(min_cnt=1))
]

sequence_features = [
    Sequence('genres', SequenceEncoder(sep='|', min_cnt=1)),
    Sequence('histHighRatedMovieIds', SequenceEncoder(sep='|', min_cnt=1)),
    Sequence('negHistMovieIds', SequenceEncoder(sep='|', min_cnt=1))
]

In [10]:
features, train_loader, valid_loader = create_dataloader_fn(
    number_features, category_features, sequence_features, 64, train_df, 'label', valid_df, 4)

In [11]:
def evaluation(df, dataloader):
    preds = predict(model, dataloader)
    return roc_auc_score(df['label'], preds.ravel())

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

In [13]:
din_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1)]

gru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='GRU')]

aigru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AIGRU')]

agru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AGRU')]

augru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AUGRU')]

augru_attention_groups_with_neg = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds', 'neg_hist': 'negHistMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AUGRU')]

models = [
    DNN(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    WideDeep(
        features,
        wide_features=['genres', 'movieId'],
        deep_features=['age', 'gender', 'movieId', 'occupation',
         'zipCode', 'histHighRatedMovieIds'],
        cross_features=[('movieId', 'histHighRatedMovieIds')],
        num_classes=2, embedding_size=16, hidden_layers=(32, 16),
        final_activation='sigmoid', dropout=0.3),
    DeepFM(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIN(features, din_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIEN(features, gru_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIEN(features, aigru_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIEN(features, agru_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIEN(features, augru_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIEN(features, augru_attention_groups_with_neg, 2, 16, (32, 16),
         final_activation='sigmoid', dropout=0.3, use_negsampling=True)
]

In [14]:
scores = []
for model in models:
    print(model)
    loss_func = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    fit(10, model, loss_func, optimizer,
        train_loader, valid_loader, notebook=True, auxiliary_loss_rate=0.1)
    scores.append(evaluation(valid_df, valid_loader))

DNN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:histHighRatedMovieIds): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=113, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3)
      (dense1): Linear(in_features=32, out_features=16, bias=True)
      (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ac

WideDeep(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): SumPooling()
  (pooling:histHighRatedMovieIds): SumPooling()
  (pooling:negHistMovieIds): SumPooling()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=81, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3)
      (dense1): Linear(in_features=32, out_features=16, bias=True)
      (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

DeepFM(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (embedding:age): Linear(in_features=1, out_features=16, bias=False)
  (pooling:genres): MaxPooling()
  (pooling:histHighRatedMovieIds): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (linear): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=1, out_features=1, bias=True)
      (batchnorm0): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (fm): FM()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=129, out_features=32, bias=True)
      (batchnorm0): BatchNorm

DIN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Attention(
    (mlp): MLP(
      (_sequential): Sequential(
        (dense0): Linear(in_features=64, out_features=16, bias=True)
        (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation0): PReLU(num_parameters=1)
        (dropout0): Dropout(p=0.1)
        (dense1): Linear(in_features=16, out_features=8, bias=True)
        (batchnorm1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_runnin

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (interest_evolution): GRU(16, 16, batch_first=True)
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation0): PReLU(num_parameters=1)
          (dropout0): Dropout(p=0.1)
   

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation0): PReLU(num_parameters=1)
          (dropout0): Dropout(p=0.1)
          (dense1): Linear(in_features=16, out_features=8, 

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation0): PReLU(num_parameters=1)
          (dropout0): Dropout(p=0.1)
          (dense1): Linear(in_features=16, out_features=8, 

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation0): PReLU(num_parameters=1)
          (dropout0): Dropout(p=0.1)
          (dense1): Linear(in_features=16, out_features=8, 

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2917, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (auxiliary_net): AuxiliaryNet(
      (_sequential): Sequential(
        (dense0): Linear(in_features=32, out_features=100, bias=True)
        (activation0): Sigmoid()
        (dense1): Linear(in_features=100, out_features=50, bias=True)
        (activation1): Sigmoid()
        (final_layer): Linear(in_features=50, out_features=1, bias=True)
      )
    )
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
 

In [15]:
print(scores)

[0.5678126351433473, 0.561144856709187, 0.5784686439168888, 0.5822279255266528, 0.5906340183893881, 0.5961033491524658, 0.601482231034707, 0.5828921594952953, 0.6221356676681995]


# share embeddings

In [16]:
features.category_features[1].name

'movieId'

In [17]:
word2idx = features.category_features[1].column_flow.transformers[0].word2idx
idx2word = features.category_features[1].column_flow.transformers[0].idx2word

In [18]:
number_features = [
    Number('age', StandardScaler())
]

category_features = [
    Category('gender', CategoryEncoder(min_cnt=1)),
    Category('movieId',
             CategoryEncoder(word2idx=word2idx, idx2word=idx2word),
             embedding_name='movieId'),
    Category('occupation', CategoryEncoder(min_cnt=1)),
    Category('zipCode', CategoryEncoder(min_cnt=1))
]

sequence_features = [
    Sequence('genres', SequenceEncoder(sep='|', min_cnt=1)),
    Sequence('histHighRatedMovieIds',
             SequenceEncoder(sep='|', word2idx=word2idx, idx2word=idx2word),
             embedding_name='movieId'),
    Sequence('negHistMovieIds',
             SequenceEncoder(sep='|', word2idx=word2idx, idx2word=idx2word),
             embedding_name='movieId')
]

features, train_loader, valid_loader = create_dataloader_fn(
    number_features, category_features, sequence_features, 64, train_df, 'label', valid_df, 4)

In [19]:
din_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1)]

gru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='GRU')]

aigru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AIGRU')]

agru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AGRU')]

augru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AUGRU')]

augru_attention_groups_with_neg = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds', 'neg_hist': 'negHistMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AUGRU')]

models = [
    DNN(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    WideDeep(
        features,
        wide_features=['genres', 'movieId'],
        deep_features=['age', 'gender', 'movieId', 'occupation',
         'zipCode', 'histHighRatedMovieIds'],
        cross_features=[('movieId', 'histHighRatedMovieIds')],
        num_classes=2, embedding_size=16, hidden_layers=(32, 16),
        final_activation='sigmoid', dropout=0.3),
    DeepFM(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIN(features, din_attention_groups, 2, 16, (32, 16),
        final_activation='sigmoid', dropout=0.3),
    DIEN(features, gru_attention_groups, 2, 16, (32, 16),
         final_activation='sigmoid', dropout=0.3),
    DIEN(features, aigru_attention_groups, 2, 16, (32, 16),
         final_activation='sigmoid', dropout=0.3),
    DIEN(features, agru_attention_groups, 2, 16, (32, 16),
         final_activation='sigmoid', dropout=0.3),
    DIEN(features, augru_attention_groups, 2, 16, (32, 16),
         final_activation='sigmoid', dropout=0.3),
    DIEN(features, augru_attention_groups_with_neg, 2, 16, (32, 16),
         final_activation='sigmoid', dropout=0.3, use_negsampling=True)
]

In [20]:
scores = []
for model in models:
    print(model)
    loss_func = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    fit(10, model, loss_func, optimizer,
        train_loader, valid_loader, notebook=True, auxiliary_loss_rate=0.1)
    scores.append(evaluation(valid_df, valid_loader))

DNN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:histHighRatedMovieIds): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=113, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3)
      (dense1): Linear(in_features=32, out_features=16, bias=True)
      (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation1): ReLU()
      (dropout1): Dropout(p=0.3)
    )
  )
  (final_layer): Linear(in_features=16, out_features=1, bias=True)
)


WideDeep(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): SumPooling()
  (pooling:histHighRatedMovieIds): SumPooling()
  (pooling:negHistMovieIds): SumPooling()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=81, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3)
      (dense1): Linear(in_features=32, out_features=16, bias=True)
      (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation1): ReLU()
      (dropout1): Dropout(p=0.3)
    )
  )
  (final_layer): Linear(in_features=64, out_features=1, bias=True)
)


DeepFM(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:age): Linear(in_features=1, out_features=16, bias=False)
  (pooling:genres): MaxPooling()
  (pooling:histHighRatedMovieIds): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (linear): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=1, out_features=1, bias=True)
      (batchnorm0): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (fm): FM()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=129, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3)
      

DIN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Attention(
    (mlp): MLP(
      (_sequential): Sequential(
        (dense0): Linear(in_features=64, out_features=16, bias=True)
        (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation0): PReLU(num_parameters=1)
        (dropout0): Dropout(p=0.1)
        (dense1): Linear(in_features=16, out_features=8, bias=True)
        (batchnorm1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation1): PReLU(num_parameters=1)
        (dropout1): Dropout(p=0.1)
      )
    )
    (fc): Linear(in_features

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (interest_evolution): GRU(16, 16, batch_first=True)
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation0): PReLU(num_parameters=1)
          (dropout0): Dropout(p=0.1)
          (dense1): Linear(in_features=16, out_features=8, bias=True)
          (batchnorm1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation0): PReLU(num_parameters=1)
          (dropout0): Dropout(p=0.1)
          (dense1): Linear(in_features=16, out_features=8, bias=True)
          (batchnorm1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation1):

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation0): PReLU(num_parameters=1)
          (dropout0): Dropout(p=0.1)
          (dense1): Linear(in_features=16, out_features=8, bias=True)
          (batchnorm1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation1):

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation0): PReLU(num_parameters=1)
          (dropout0): Dropout(p=0.1)
          (dense1): Linear(in_features=16, out_features=8, bias=True)
          (batchnorm1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation1):

DIEN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1718, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2957, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (attention_pooling:group1): Interest(
    (interest_extractor): GRU(16, 16, batch_first=True)
    (auxiliary_net): AuxiliaryNet(
      (_sequential): Sequential(
        (dense0): Linear(in_features=32, out_features=100, bias=True)
        (activation0): Sigmoid()
        (dense1): Linear(in_features=100, out_features=50, bias=True)
        (activation1): Sigmoid()
        (final_layer): Linear(in_features=50, out_features=1, bias=True)
      )
    )
    (attention): Attention(
      (mlp): MLP(
        (_sequential): Sequential(
          (dense0): Linear(in_features=64, out_features=16, bias=True)
          (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, aff

In [21]:
print(scores)

[0.5732876189614773, 0.5810577431308315, 0.5488579415530639, 0.5971491643371369, 0.5990090194493359, 0.5756873408311688, 0.6166408981573868, 0.5890624690848553, 0.6236704721148927]
